## Analizando frecuencia de palabras

Se realizan los siguientes pasos:

- Cargar dataframe de noticias.
- Cargar los cuerpos de noticia preprocesados.
- Encontrar la frecuencia de palabras clave de la categoría "Hecho contra la población civil" para cada cuerpo de noticia preprocesado.
- Crear el ranking de acuerdo a la suma de las frecuencias.

In [49]:
import numpy as np
import pandas as pd
import sqlite3
from nltk import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

def show_news(n, df):
    fila = df[df['ID']==n]
    print(
        "Suma:", fila['suma_freq'].values[0],
        "\nRanking:", fila['Ranking'].values[0],
        "\nTitulo:", fila['titulo'].values[0],
        "\nNumero palabras:", fila['long_cuerpo'].values[0],
        "\nDiario:", fila['diario'].values[0],
        "\nCuerpo:", fila['cuerpo'].values[0],
        "\nurl:", fila['url'].values[0],
        "\nID:", fila['ID'].values[0],
    )

-----------

**Cargando dataframe de noticias**
- Se encuentra la longitud del cuerpo de la noticia.
- Se asigna un ID a cada noticia.
- Se eliminan las noticias cuyo cuerpo no tiene ninguna palabra.

In [2]:
cnx = sqlite3.connect('../data/data.db')
df = pd.read_sql_query("SELECT * FROM news", cnx)
df['fecha_publicacion'] = pd.to_datetime(df['fecha_publicacion'])
df['long_cuerpo'] = df['cuerpo'].apply(lambda x: len(str(x).split()))
df['ID'] = df.index
df = df[df['long_cuerpo']!=0].reset_index()
df.head(2)

,index,titulo,cuerpo,fecha_publicacion,diario,url,page,long_cuerpo,ID
0,0,Campesino asesinado en Baraya,Un impacto en el pecho con una escopeta calibr...,2020-06-11 17:00:33,lanacion,https://www.lanacion.com.co/campesino-asesinad...,2,107,0
1,1,‘Morcilla’ a responder por homicidio de un jov...,Jorge Leonardo Pájaro Paredes conocido como ‘M...,2020-06-10 10:10:56,lanacion,https://www.lanacion.com.co/morcilla-a-respond...,2,228,1



**Cargando dataframe de cuerpo de noticia preprocesado**

- Se carga el archivo.
- Se pega al dataframe de noticias.

In [3]:
serie_word = pd.read_csv('serie_word.csv')['text']
print(serie_word.shape)
serie_word.head(2)

(19284,)


0    campesin asesin baray impact pech escopet cali...
1    morcill respond homicidi jov pajar conoc morci...
Name: text, dtype: object

In [4]:
print("Tamanhos antes:", df.shape, serie_word.shape)
df['Preprocesado'] = serie_word.values

Tamanhos antes: (19284, 9) (19284,)


---------

**Encontrando la frecuencia de palabras clave de la categoría "Hecho contra la población civil" para cada cuerpo de noticia preprocesado**

- Se genera la lista de palabras clave y se preprocesa.
- Se eliminan las noticias que no tienen ninguna de las palabras clave.
- Se aplica el CountVectorizer sobre el dataframe de noticias para encontrar las frecuencias.

In [8]:
searchfor = ["homicidio",
             "masacre", 
             "secuestro",
             "tortura",
             "desplazamiento",
             "forzado",
             "confinamiento",
             "accidente",
             "minas",
             "antipersonal",
             "atentado",
             "reclutamiento",
             #"utilizacion NNA",
             "violencia",
             "sexual",
             "ataque",
             "indiscriminado",
             #"ataque",
             "bienes",
             "civiles"]

tam = len(searchfor)

searchfor = pd.Series(searchfor)

spanishstemmer = SnowballStemmer('spanish')
print('Stemming words...')
# Stemming (word root)
searchfor = searchfor.apply(lambda x: " ".join(spanishstemmer.stem(word) for word in x.split()))
searchfor

Stemming words...


0         homicidi
1           masacr
2         secuestr
3           tortur
4          desplaz
5             forz
6           confin
7         accident
8              min
9     antipersonal
10           atent
11          reclut
12        violenci
13          sexual
14           ataqu
15     indiscrimin
16            bien
17           civil
dtype: object

In [6]:
searchfor_list = list(searchfor)
df['OK'] = df['Preprocesado'].str.contains('|'.join(searchfor_list))
df = df[df['OK'] == True]
print("Tamanhos despues:", df.shape)
df.head(2)

Tamanhos despues: (15115, 11)


,index,titulo,cuerpo,fecha_publicacion,diario,url,page,long_cuerpo,ID,Preprocesado,OK
0,0,Campesino asesinado en Baraya,Un impacto en el pecho con una escopeta calibr...,2020-06-11 17:00:33,lanacion,https://www.lanacion.com.co/campesino-asesinad...,2,107,0,campesin asesin baray impact pech escopet cali...,True
1,1,‘Morcilla’ a responder por homicidio de un jov...,Jorge Leonardo Pájaro Paredes conocido como ‘M...,2020-06-10 10:10:56,lanacion,https://www.lanacion.com.co/morcilla-a-respond...,2,228,1,morcill respond homicidi jov pajar conoc morci...,True


In [7]:
vectorizer = CountVectorizer(vocabulary=searchfor)
df['Freqs'] = df['Preprocesado'].apply(lambda x: vectorizer.fit_transform([x]).toarray())
df['Freqs'].head(2)

0    [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
1    [[5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
Name: Freqs, dtype: object

---------

**Creando el ranking de acuerdo a la suma de las frecuencias**

In [38]:
df['suma_freq'] = df.apply(lambda x: x['Freqs'].sum(), axis=1)
df['Ranking'] = df['suma_freq']/df['long_cuerpo']
df.head(2)

,index,titulo,cuerpo,fecha_publicacion,diario,url,page,long_cuerpo,ID,Preprocesado,OK,Freqs,bool_freqs,Ranking1,suma_freq,Ranking
0,0,Campesino asesinado en Baraya,Un impacto en el pecho con una escopeta calibr...,2020-06-11 17:00:33,lanacion,https://www.lanacion.com.co/campesino-asesinad...,2,107,0,campesin asesin baray impact pech escopet cali...,True,"[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.055556,1,0.009346
1,1,‘Morcilla’ a responder por homicidio de un jov...,Jorge Leonardo Pájaro Paredes conocido como ‘M...,2020-06-10 10:10:56,lanacion,https://www.lanacion.com.co/morcilla-a-respond...,2,228,1,morcill respond homicidi jov pajar conoc morci...,True,"[[5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.111111,6,0.026316


In [39]:
aux = df[['Ranking', 'ID']].reset_index()
aux.sort_values(by='Ranking', ascending=False, inplace=True)
print(aux.head(2))
print(df[df['ID']==15539]['suma_freq'])

       index   Ranking     ID
10419  13567  0.333333  13631
8147   10713  0.142857  10756
15472    38
Name: suma_freq, dtype: int64


In [40]:
show_news(13631, df)

Suma: 1 
Ranking: 0.3333333333333333 
Titulo: Falleció motociclista accidentado el pasado domingo 
Numero palabras: 3 
Diario: lanacion 
Cuerpo: Llevaba la vía 
url: https://www.lanacion.com.co/fallecio-motociclista-accidentado-el-pasado-domingo/ 
ID: 13631


In [41]:
show_news(10756, df)

Suma: 1 
Ranking: 0.14285714285714285 
Titulo: Menor de edad protagonizó aparatoso accidente en el sur de Neiva 
Numero palabras: 7 
Diario: lanacion 
Cuerpo: * Con información de OLE MI DIARIO.  
url: https://www.lanacion.com.co/menor-de-edad-protagonizo-aparatoso-accidente-en-el-sur-de-neiva/ 
ID: 10756


----------
**Verificando que el conteo de frecuencias haya sido correcto**

In [175]:
aux = df[df['ID']==15539]['Freqs'].values[0][0]
print(aux)

[ 0  0  0  0  2  0  0  0  0  0  0  0 27  7  0  0  1  1]


In [179]:
corpus = [df[df['ID']==15539]['Preprocesado'].values[0]]
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(X.toarray())
print(corpus[0].count('violenci'))

['homicidi', 'masacr', 'secuestr', 'tortur', 'desplaz', 'forz', 'confin', 'accident', 'min', 'antipersonal', 'atent', 'reclut', 'violenci', 'sexual', 'ataqu', 'indiscrimin', 'bien', 'civil']
[[ 0  0  0  0  2  0  0  0  0  0  0  0 27  7  0  0  1  1]]
27


-------
**Conclusiones preliminares**

- Las noticias con mayor ranking son las noticias más largas.
- No todas parecen servir para identificar un hecho contra la población civil.
- Hay que incluir algún tipo de peso para las palabras.

In [34]:
def boolean_freq(lista):
    lista = list(lista)[0]
    # print(lista)
    aux = []
    for i in range(len(lista)):
        valor = 0
        if lista[i] > 0:
            valor = 1
        aux.append(valor)
    return aux

In [50]:
df['bool_freqs'] = df['Freqs'].apply(lambda x: boolean_freq(x))
df['Ranking'] = df['bool_freqs'].apply(lambda x: np.sum(x)/tam)
df.head(2)

,index,titulo,cuerpo,fecha_publicacion,diario,url,page,long_cuerpo,ID,Preprocesado,OK,Freqs,bool_freqs,Ranking1,suma_freq,Ranking
0,0,Campesino asesinado en Baraya,Un impacto en el pecho con una escopeta calibr...,2020-06-11 17:00:33,lanacion,https://www.lanacion.com.co/campesino-asesinad...,2,107,0,campesin asesin baray impact pech escopet cali...,True,"[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.055556,1,0.055556
1,1,‘Morcilla’ a responder por homicidio de un jov...,Jorge Leonardo Pájaro Paredes conocido como ‘M...,2020-06-10 10:10:56,lanacion,https://www.lanacion.com.co/morcilla-a-respond...,2,228,1,morcill respond homicidi jov pajar conoc morci...,True,"[[5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.111111,6,0.111111


In [51]:
aux = df[['Ranking', 'ID']].reset_index()
aux.sort_values(by='Ranking', ascending=False, inplace=True)
print(aux.head(3))
print(df[df['ID']==15539]['suma_freq'])

       index   Ranking     ID
2276    3575  0.444444   3597
14981  19118  0.444444  19187
390      532  0.388889    532
15472    38
Name: suma_freq, dtype: int64


In [52]:
show_news(3597, df)

Suma: 13 
Ranking: 0.4444444444444444 
Titulo: Asesinan a guardia indígena en zona turística del Chocó 
Numero palabras: 504 
Diario: hoydiariodelmagdalena 
Cuerpo:  		
				
				
		Según la entidad, Anuar Rojas Isaramá fue asesinado en Agua Blanca, que pertenece a Nuquí, un población frecuentada por turistas nacionales y extranjeros por la belleza de sus playas, variada gastronomía y posibilidades de ecoturismo, entre otros atractivos. “Rechazamos asesinato del guardia indígena Anuar Rojas Isaramá en Agua Blanca (…) Solicitamos su esclarecimiento”, escribió la Defensoría en Twitter. Añadió: “Tras el hecho, 80 personas pertenecientes a la comunidad están en proceso de desplazamiento forzado.  Pedimos activar las rutas institucionales de atención”. En el departamento del Chocó, fronterizo con Panamá, operan las guerrillas del Ejército de Liberación Nacional (ELN), las paramilitares Autodefensas Gaitanistas de Colombia (AGC) y otras bandas criminales que amenazan y ponen en riesgo constan

In [53]:
show_news(19187, df)

Suma: 25 
Ranking: 0.4444444444444444 
Titulo: ¿Por qué las Farc continúan ensañadas con Pradera, Valle? 
Numero palabras: 1164 
Diario: elpais 
Cuerpo: Tras el atentado con motobomba, autoridades y analistas coinciden en que el municipio aún tiene un valor estratégico para la guerrilla.Pradera es el municipio del Valle del Cauca más afectado por la violencia de los grupos ilegales. En menos de dos años, esta ciudad, ubicada en las estribaciones de la cordillera Central, ha sido blanco de dos atentados terroristas con bombas que han dejado casi un centenar de heridos.  Una mezcla de la influencia de la columna Gabriel Galvis de las Farc y de dos poderosas bandas delincuenciales han creado el pánico en esta población, afectada no solo por los ataques guerrilleros, sino por las minas antipersonales sembradas en su área rural y por las extorsiones. El atentado más reciente ocurrió   el jueves pasado frente al edificio de la Alcaldía y a la estación de Policía de Pradera, cuando dos hombre

In [55]:
show_news(532, df)

Suma: 18 
Ranking: 0.3888888888888889 
Titulo: Condenados 30 paramilitares por masacres en Cesar y Norte de Santander 
Numero palabras: 379 
Diario: hoydiariodelmagdalena 
Cuerpo:   La sentencia, que también afecta a 29 postulados más, está motivada en la investigación de una fiscalía adscrita a la dirección de justicia transicional, que logró documentar 465 hechos delictivos, entre los que reposan los delitos de homicidio, desaparición forzada, desplazamiento forzado, reclutamiento ilícito de menores, violencia basada en género, entre otros. De acuerdo con las pruebas de la fiscalía, se conoció que el frente héctor julio peinado becerra delinquió desde el año 1992 hasta marzo de 2006 en los municipios del sur de cesar, como san alberto, san martín, aguachica, gamarra y río de oro, así como también en ocaña, abrego y la playa de belén (norte de santander). A ‘juancho prada’ y su estructura paramilitar se le atribuyeron el homicidio de rosalba contreras de castro, quien fungía como secr